In [1]:
import os
import pandas as pd

In [2]:
df_MB_full = pd.read_csv('./IntermediateProducts/Result_ALL.csv')
df_MB_select_org = pd.read_csv('./IntermediateProducts/Result_Selected_Organisms.csv')

In [3]:
df_NLSdb = pd.read_csv('./SourceData/NLSdb/signals.csv')

In [30]:
# df_MB_full.head()
# df_MB_select_org.head()
df_NLSdb.head()

,ID,Sequence,SignalType,AnnotationType,ConfidenceNuc,ConfidenceFam,Origin,Added,Modified
0,1,AAKKPR,NLS,Potential,2,2,In Silico Mutagenesis,2017-10-11 15:10:55,2017-10-11 15:10:55
1,2,AAKKQK,NLS,Potential,3,3,In Silico Mutagenesis,2017-10-11 15:10:55,2017-10-11 15:10:55
2,3,AAKKTK,NLS,Potential,2,2,In Silico Mutagenesis,2017-10-11 15:10:55,2017-10-11 15:10:55
3,4,AAKRRL,NLS,Potential,3,3,In Silico Mutagenesis,2017-10-11 15:10:55,2017-10-11 15:10:55
4,5,AEKKAK,NLS,Potential,5,3,In Silico Mutagenesis,2017-10-11 15:10:55,2017-10-11 15:10:55


In [31]:
df_NLSdb.shape

(2651, 9)

#### Make a list of NLSs

In [7]:
list_NLS = df_NLSdb['Sequence'].unique()
print(len(list_NLS))

2651


## 2. Search for proteins with putative NLSs

### A funciton to find NLS(s) in a given protein sequence

In [8]:
def searchNLS(protein_seq):
    NLS_positions = dict()
    for i, NLS in enumerate(list_NLS):
        position = protein_seq.find(NLS)
        if position != -1:
            NLS_positions[NLS] = position
        
    return NLS_positions

In [8]:
def searchNLS_2(protein_seq):
    NLS_positions = dict()
    for i, NLS in enumerate(list_NLS):
        position = protein_seq.find(NLS)
        if position != -1:
            NLS_positions[NLS] = position
        
    return NLS_positions

### Application of searchNLS to sequence

In [10]:
df_MB_select_org['NLS locations'] = df_MB_select_org['AA_sequence'].apply(searchNLS)

In [11]:
df_MB_select_org.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3240 entries, 0 to 3239
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Uniprot_ID     3240 non-null   object
 1   Organism       3240 non-null   object
 2   Protein_name   3240 non-null   object
 3   AH_or_Not      3240 non-null   object
 4   AA_sequence    3240 non-null   object
 5   Prediction     3240 non-null   object
 6   NLS locations  3240 non-null   object
dtypes: object(7)
memory usage: 177.3+ KB


In [12]:
df_MB_select_org_NLS_positive = df_MB_select_org[df_MB_select_org['NLS locations'] != {}]

In [13]:
df_MB_select_org_NLS_positive.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 219 entries, 11 to 3209
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Uniprot_ID     219 non-null    object
 1   Organism       219 non-null    object
 2   Protein_name   219 non-null    object
 3   AH_or_Not      219 non-null    object
 4   AA_sequence    219 non-null    object
 5   Prediction     219 non-null    object
 6   NLS locations  219 non-null    object
dtypes: object(7)
memory usage: 13.7+ KB


In [19]:
size_NLS = df_MB_select_org_NLS_positive.groupby('Organism').size()
size_whole = df_MB_select_org.groupby('Organism').size()

In [25]:
mydf = pd.DataFrame({'NLS': pd.Series(size_NLS), 'Whole': pd.Series(size_whole)})

In [27]:
mydf['Ratio'] = mydf['NLS'] / mydf['Whole']

In [29]:
mydf_major = mydf[mydf['Whole'] > 10]
mydf_major

,NLS,Whole,Ratio
Organism,,,
Arabidopsis thaliana (Mouse-ear cress),59.0,717,0.082287
Bos taurus (Bovine),17.0,195,0.087179
Caenorhabditis elegans,16.0,346,0.046243
Canis lupus familiaris (Dog) (Canis familiaris),2.0,22,0.090909
Danio rerio (Zebrafish) (Brachydanio rerio),10.0,102,0.098039
Drosophila melanogaster (Fruit fly),9.0,159,0.056604
Gallus gallus (Chicken),3.0,46,0.065217
Homo sapiens (Human),57.0,815,0.069939
Macaca fascicularis (Crab-eating macaque) (Cynomolgus monkey),1.0,23,0.043478
